In [ ]:
# Dependecies

In [ ]:
#Import CSV/API

In [ ]:
#Print Original Dataframe

In [ ]:
#Find valuable data using lists

In [ ]:
#Loop Through Info

In [ ]:
#Merge/Rename columns if any overlap with different data

In [ ]:
#Add columns to generate more specific data that could be useful for analysis

In [ ]:
#Add Bins putting data into catagories if numbers are involved

In [ ]:
# Find total/average data - use indexes

In [ ]:
#Sort Data 

In [ ]:
#Find amount of tweets/social media used to cover the topic chosen

In [ ]:
#Use API url/query to gather information to determine other data

In [ ]:
#Create Dataframe for Data showing the type of data, when it occured, how it occured, etc.

In [ ]:
#Plot Pie Graph to show percentages of Data

In [ ]:
#Plot Bar Graph/Line Graph/Scatter to show correlations between Data 

In [ ]:
#Explain trend(Education level vs. Unemployment level per zones)
#Explain trend (Education level vs. Median Age)
#Explain trend (Education level vs. Household income)
#Explain trend 

In [ ]:
#Export Graph images